In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time
import math
from script.rs_pca import *

In [2]:


def usporedi (
    df,
    column,
    ax = None,
    args = (tuple(), tuple()),
    kwargs = (
        {'density' : True, 'alpha' : 0.5},
        {'density' : True, 'alpha' : 0.5}
    )
):
    if ax is None:
        ax = plt.gca()

    if df[column].dtype.name == 'category' or df[column].dtype.name == 'object':
        kwargs0, kwargs1 = (copy.deepcopy(kwargs[0]), copy.deepcopy(kwargs[1]))
        dens0, dens1 = (
            kwargs0.pop('density', False),
            kwargs1.pop('density', False)
        )

        nonnulls = pd.notnull(df[column])

        categories = np.sort(df.loc[nonnulls, column].unique())

        ax.bar(
            categories,
            np.array(
                [
                    ((df[column] == category) & ~df.PRIJEVREMENI_RASKID).sum()
                        for category in categories
                ],
                dtype = float
            ) / (nonnulls & ~df.PRIJEVREMENI_RASKID).sum() if dens0 else [
                ((df[column] == category) & ~df.PRIJEVREMENI_RASKID).sum()
                    for category in categories
            ],
            *args[0],
            **kwargs0
        )
        ax.bar(
            categories,
            np.array(
                [
                    ((df[column] == category) & df.PRIJEVREMENI_RASKID).sum()
                        for category in categories
                ],
                dtype = float
            ) / (nonnulls & df.PRIJEVREMENI_RASKID).sum() if dens1 else [
                ((df[column] == category) & df.PRIJEVREMENI_RASKID).sum()
                    for category in categories
            ],
            *args[1],
            **kwargs1
        )
    else:
        ax.hist(df.loc[~df.PRIJEVREMENI_RASKID, column], *args[0], **kwargs[0])
        ax.hist(df.loc[df.PRIJEVREMENI_RASKID, column], *args[1], **kwargs[1])

In [20]:
##  UCITAVANJE SKUPA PODATAKA
##
##  S obzirom na to da je skup podataka vrlo velik (5 193 124 redaka), mjerit
##  cemo i vrijeme potrebno za njegovo ucitavanje.  Podatci bi se, medutim, iz
##  CSV datoteke puno brze ucitali da se datumi ne parsiraju, nego da ostanu
##  stringovi, ali vjerojatno ce nam datumi ionako biti potrebni u datumskom
##  obliku pa ih je bolje odmah takve i ucitati.  Stoga, jednom kada se CSV
##  ucita, sprema se pickle cijelog skupa podataka tako da svaki sljedeci put
##  ucitavanje bude puno brze (ako datoteka s pickle-om postoji, automatski se
##  skup podataka cita iz nje; CSV se cita tek ako datoteka s pickle-om ne
##  postoji).

# Pocni mjeriti vrijeme.
t0 = time.time()


# Ucitaj skup podataka.  Prvo se pokusava ucitati pickle iz datoteke
# `training_dataset_enc.pkl', a, ako ta datoteka ne postoji, onda se ucitava
# tablica u CSV obliku iz `training_dataset_enc.csv' i odmah se sprema pickle u
# datoteku `training_dataset_enc.pkl' za brze ucitavanje ubuduce.
df = None
try:
    df = pd.read_pickle('../data/spljosteni_i_dropani_trening_set.pkl')
except FileNotFoundError:
    df = pd.read_csv(
        '../data/spljosteni_i_dropani_trening_set.csv',
        index_col = 0,
        parse_dates = [
            'DATUM_OTVARANJA',
            'PLANIRANI_DATUM_ZATVARANJA'
        ],
        infer_datetime_format = True
    )
    df.reset_index(drop = True, inplace = True)
    df.KLIJENT_ID = df.KLIJENT_ID.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(
                df.KLIJENT_ID[pd.notnull(df.KLIJENT_ID)].unique()
            ),
            ordered = False
        )
    )
    df.OZNAKA_PARTIJE = df.OZNAKA_PARTIJE.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(
                df.OZNAKA_PARTIJE[pd.notnull(df.OZNAKA_PARTIJE)].unique()
            ),
            ordered = False
        )
    )
    df.VALUTA = df.VALUTA.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(df.VALUTA[pd.notnull(df.VALUTA)].unique()),
            ordered = False
        )
    )
    df.VRSTA_KLIJENTA = df.VRSTA_KLIJENTA.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(
                df.VRSTA_KLIJENTA[pd.notnull(df.VRSTA_KLIJENTA)].unique()
            ),
            ordered = False
        )
    )
    df.PROIZVOD = df.PROIZVOD.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(df.PROIZVOD[pd.notnull(df.PROIZVOD)].unique()),
            ordered = False
        )
    )
    df.VRSTA_PROIZVODA = df.VRSTA_PROIZVODA.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(
                df.VRSTA_PROIZVODA[pd.notnull(df.VRSTA_PROIZVODA)].unique()
            ),
            ordered = False
        )
    )
    df.TIP_KAMATE = df.TIP_KAMATE.astype(
        pd.api.types.CategoricalDtype(
            categories = np.sort(
                df.TIP_KAMATE[pd.notnull(df.TIP_KAMATE)].unique()
            ),
            ordered = False
        )
    )
    df.to_pickle('../data/spljosteni_i_dropani_trening_set.pkl')

# Zavrsi mjerenje vremena.
t1 = time.time()

# Izracunaj vremenski period od t0 do t1 u sekundama.
d = float(t1 - t0)

# Ispisi vrijeme.
print(
    'Trajanje ucitavanja: {h:d}h {m:02d}m {s:06.3f}s ({S:.3f}s)'.format(
        h = int(math.floor(d / 3600)),
        m = int(math.floor((d - 3600 * int(math.floor(d / 3600))) / 60)),
        s = d - 60 * int(math.floor(d / 60)),
        S = d
    )
)

# Oslobodi memoriju.
del d
del t0
del t1


Trajanje ucitavanja: 0h 00m 00.041s (0.041s)


In [ ]:
(df-df.mean())/df.std()

In [21]:
#numerifikacija
df['DATUM_OTVARANJA'] = pd.to_numeric(df['DATUM_OTVARANJA'])
df['PLANIRANI_DATUM_ZATVARANJA'] = pd.to_numeric(df['PLANIRANI_DATUM_ZATVARANJA'])/(10**16)
#normalizacija
df['DATUM_OTVARANJA'] = (df['DATUM_OTVARANJA'] - df['DATUM_OTVARANJA'].mean())/df['DATUM_OTVARANJA'].std()
df['PLANIRANI_DATUM_ZATVARANJA'] = (df['PLANIRANI_DATUM_ZATVARANJA'] - df['PLANIRANI_DATUM_ZATVARANJA'].mean())/df['PLANIRANI_DATUM_ZATVARANJA'].std()

In [22]:
df.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,STAROST,UGOVORENI_IZNOS,VISINA_KAMATE,VRSTA_KLIJENTA,VRSTA_PROIZVODA,PROIZVOD,VALUTA,TIP_KAMATE,PRIJEVREMENI_RASKID
0,1588,256644,-5.166776,-1.212613,46,NaN,2.7,1410,L,TM0109,2,C,0
1,1588,256644,-5.166776,-1.144293,47,NaN,1.9,1410,L,TM0109,2,C,0
2,1588,256644,-5.166776,-1.075238,46,NaN,1.9,1410,L,TM0109,2,C,0
3,1588,256644,-5.166776,-1.028221,46,NaN,1.9,1410,L,TM0109,2,C,0
4,1145,293918,-5.069029,-1.073768,40,71552.0,4.0,1610,A,FL1306,4,C,0


In [38]:
(df.loc[pd.isna(df.UGOVORENI_IZNOS),:]).shape

(421, 13)

In [42]:
(df.loc[pd.isna(df.UGOVORENI_IZNOS),:]).VRSTA_PROIZVODA.dtype.categories

for c in (df.loc[pd.isna(df.UGOVORENI_IZNOS),:]).columns:
    try:
        print((df.loc[pd.isna(df.UGOVORENI_IZNOS),:])[c].dtype.categories)
    except:
        pass

Int64Index([      1002,       1013,       1025,       1028,       1041,
                  1044,       1045,       1046,       1061,       1063,
            ...
               1848491,    1848649,    1848662,    1848786,    1848992,
               1849524,    1849542,    1850220,    1861085, 9991410999],
           dtype='int64', length=253336)
Int64Index([   163007,    163116,    163312,    163577,    163718,    163825,
               164109,    164426,    164527,    164971,
            ...
            113528889, 113528901, 113528907, 113528923, 113528926, 113528960,
            113528975, 113528976, 113528993, 113529034],
           dtype='int64', length=547138)
Int64Index([1110, 1120, 1210, 1310, 1320, 1330, 1350, 1410, 1420, 1430, 1510,
            1550, 1610],
           dtype='int64')
Index(['A', 'L'], dtype='object')
Index(['FD0100', 'FD0200', 'FE0100', 'FE0200', 'FI0500', 'FL0201', 'FL0202',
       'FL0600', 'FL0801', 'FL0802', 'FL0803', 'FL0804', 'FL0900', 'FL0901',
       'FL1

In [33]:
(df.loc[pd.isna(df.UGOVORENI_IZNOS),:].PRIJEVREMENI_RASKID).sum()

18

In [37]:
(df.loc[pd.isna(df.UGOVORENI_IZNOS),:].VRSTA_PROIZVODA == 'L').sum()

421

In [26]:
df.describe()

,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,STAROST,UGOVORENI_IZNOS,VISINA_KAMATE,PRIJEVREMENI_RASKID
count,1.154519e+06,1.154519e+06,1.154519e+06,1.154098e+06,1.152727e+06,1.154519e+06
mean,1.364907e-12,-2.165864e-12,4.943884e+01,2.417846e+05,3.643697e+00,1.926820e-01
std,1.000000e+00,1.000000e+00,1.681399e+01,3.332791e+06,3.508003e+00,3.944056e-01
min,-5.166776e+00,-4.338461e+00,-7.000000e+00,1.000000e+00,-4.000000e-03,0.000000e+00
25%,-5.488650e-01,-6.873532e-01,3.700000e+01,2.489129e+04,1.100000e+00,0.000000e+00
50%,9.429122e-02,-1.900092e-01,4.900000e+01,5.900000e+04,2.600000e+00,0.000000e+00
75%,7.390902e-01,4.476492e-01,6.200000e+01,1.173000e+05,6.200000e+00,0.000000e+00
max,1.679593e+00,8.899560e+00,1.050000e+02,6.055048e+08,7.200000e+01,1.000000e+00


In [ ]:
cat_df = df[['VRSTA_KLIJENTA', 'VRSTA_PROIZVODA', 'PROIZVOD', 'VALUTA', 'TIP_KAMATE']]

In [ ]:
pca = RS_PCA()

In [ ]:
pca.fit(cat_df, n_simultaneous_instances=50_000)

In [ ]:
np.cumsum(pca.explained_variance_ratios)

In [ ]:
y = pca.transform(cat_df.sample(50_000), 20)

In [ ]:
plt.scatter(y[:,0],y[:,1])

In [ ]:
pca.shape

In [ ]:
coef,valeuses = pca.reinterpret(10)

In [ ]:
coef

In [ ]:
valeuses

In [ ]:
cat_df = cat_df.copy()

In [ ]:
df.drop(columns=cat_df.columns, inplace=True)

In [ ]:
df=pd.concat([df, pd.DataFrame(pca.transform(cat_df, 20))], axis=1)

In [ ]:
df = pd.concat([df.drop(columns='PRIJEVREMENI_RASKID'), df.PRIJEVREMENI_RASKID], axis=1)

In [ ]:
import pickle

df.to_pickle('./nakon_rspca.pkl')

In [ ]:
df.info()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
df.dropna(axis=0,how='any', inplace=True)

In [ ]:
df.info()

In [ ]:
pca = PCA(n_components=3)
pca.fit(np.array(df.iloc[:,2:-1]))

In [ ]:
transformed = pca.transform(np.array(df.iloc[:,2:-1]))

In [ ]:
I = np.random.choice(transformed.shape[0], 50_000, replace = False)

In [ ]:
transformed

In [ ]:
transformed2 = transformed[I,:]

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(transformed2[:,0],transformed2[:,1],transformed2[:,2], c= df['PRIJEVREMENI_RASKID'].iloc[I])


In [ ]:
plt.scatter(transformed2[:,0], transformed2[:,1], c=df['PRIJEVREMENI_RASKID'].iloc[I])